In [1]:
import pandas as pd
import dill
import pickle

In [2]:
def create_arxiv_pubid_file():
    f = open('data/arxiv_pubid_2017.dill', 'rb')
    arxiv_pub_id = dill.load(f)
    pd.Series(arxiv_pub_id).to_csv('derived-data/arxiv_pubid_2017.csv', index=False)

In [3]:
def create_paper_venue_file():
    paper_venue_citations = pd.read_csv('data/paper_venue_citations_no_arxiv.csv', delimiter='\t', quotechar='*')

    part1 = paper_venue_citations[['paper1', 'venue1']].drop_duplicates()

    part2 = paper_venue_citations[['paper2', 'venue2']].drop_duplicates()

    part2 = part2.rename(axis='columns', mapper={'paper2' :'paper1', 'venue2':'venue1'})

    fixed = part1.append(part2)
    
    fixed.dropna(subset=['venue1'], inplace=True)
    
    fixed = fixed.drop_duplicates()

    fixed.to_csv('derived-data/venue-paper.csv', index=False)
create_paper_venue_file()

In [4]:
venue_rankings = pd.read_csv('data/h5_index_rank_quantiles_deciles_no_arxiv.tsv', delimiter='\t')
venue_rankings.head()

,venue,year,h5_index,ranking,deciles,quantiles
0,journal of symbolic logic,1949,10,1.0,1,1
1,journal of symbolic logic,1950,11,1.0,1,1
2,journal of symbolic logic,1952,10,1.0,1,1
3,operations research,1952,1,2.0,1,1
4,ieee transactions on information theory,1953,2,3.0,1,1


In [5]:
venue_rankings.columns

Index(['venue', 'year', 'h5_index', 'ranking', 'deciles', 'quantiles'], dtype='object')

In [6]:
y2015 = venue_rankings[venue_rankings.year == 2015]
#print(y2015)

print(venue_rankings[venue_rankings.venue.str.contains("web")])
#y2015["h5_index"].plot()

                                                   venue  year  h5_index  \
5501                               acm sigweb newsletter  1993         1   
6200                               acm sigweb newsletter  1994         1   
6647            international world wide web conferences  1994         7   
6975                               acm sigweb newsletter  1995         3   
7475            international world wide web conferences  1995        27   
7819                              world wide web journal  1995         1   
7840                               acm sigweb newsletter  1996         4   
8403            international world wide web conferences  1996        39   
8763                              world wide web journal  1996         9   
8784                               acm sigweb newsletter  1997         4   
9398            international world wide web conferences  1997        54   
9798                              world wide web journal  1997        10   
9818        

In [7]:
paper_venue = pd.read_csv('derived-data/venue-paper.csv')
paper_venue.head()

,paper1,venue1
0,16870052-7e2e-4bbc-beeb-21e493bea221,acm journal on computing and cultural heritage
1,ae0b4e3d-305d-4fb8-9847-60dcc9a9ec44,neurocomputing
2,6d6b0e6a-ddc6-40c9-8103-f0b31a686a76,international conference on the theory and app...
3,5d737e23-223d-4a51-8145-301588febcb1,future generation computer systems
4,4167bbcb-4ce3-4df0-9822-67d4ef340c09,international conference on conceptual structures


In [8]:
paper_venue = paper_venue.drop_duplicates()

In [9]:
authorPublicationData = pd.read_csv('derived-data/author-publications.csv')
authorPublicationData = authorPublicationData[['pub_id', 'year']]
authorPublicationData = authorPublicationData.drop_duplicates(subset='pub_id')
authorPublicationData.head()

,pub_id,year
0,001c58d3-26ad-46b3-ab3a-c1e557d16821,2011
2,001c8744-73c4-4b04-9364-22d31a10dbf1,2009
6,00338203-9eb3-40c5-9f31-cbac73a519ec,2011
11,0040b022-1472-4f70-a753-74832df65266,1998
13,005ce28f-ed77-4e97-afdc-a296137186a1,2009


In [10]:
authorPublicationData.shape

(3078230, 2)

In [11]:
publication_venues = authorPublicationData.merge(paper_venue, left_on='pub_id', right_on='paper1', how='inner')
publication_venues = publication_venues.drop('paper1', axis='columns')
publication_venues = publication_venues.rename({"venue1": "venue"}, axis='columns')

In [12]:
print(publication_venues.shape)
publication_venues.head()

(2285112, 3)


,pub_id,year,venue
0,001c58d3-26ad-46b3-ab3a-c1e557d16821,2011,visual analytics science and technology
1,001c8744-73c4-4b04-9364-22d31a10dbf1,2009,pattern recognition and machine intelligence
2,00701b05-684f-45f9-b281-425abfec482c,2011,international symposium on computer and inform...
3,00964544-cbe2-4da9-bb5a-03333160eb34,2014,international conference on computer vision an...
4,00bcf2d5-1592-46b0-81fd-933f90b5ecca,2002,international conference on conceptual structures


In [13]:
publication_venues_rank = publication_venues.merge(venue_rankings, on=['venue', 'year'], how='left')

In [14]:
publication_venues_rank.head()

,pub_id,year,venue,h5_index,ranking,deciles,quantiles
0,001c58d3-26ad-46b3-ab3a-c1e557d16821,2011,visual analytics science and technology,62.0,273.5,2.0,1.0
1,001c8744-73c4-4b04-9364-22d31a10dbf1,2009,pattern recognition and machine intelligence,23.0,1203.0,6.0,3.0
2,00701b05-684f-45f9-b281-425abfec482c,2011,international symposium on computer and inform...,18.0,1486.5,7.0,3.0
3,00964544-cbe2-4da9-bb5a-03333160eb34,2014,international conference on computer vision an...,8.0,2117.5,9.0,4.0
4,00bcf2d5-1592-46b0-81fd-933f90b5ecca,2002,international conference on conceptual structures,45.0,265.0,2.0,1.0


In [15]:
publication_venues_rank["h5_index"].max
publication_venues_rank[["h5_index", "year"]].groupby("year").mean().plot()

In [16]:
publication_venues_rank.to_csv('derived-data/publication-venues-rank.csv', index=False)